把shp文件整合，提取不同边界内部的poi类别数量。

landuse数据会溢出部分边界，所以在苏格兰和威尔士的几部分local authority会有一些数值;

19个分类，后面汇总成各种其他的共三类；

fclass分成三类，居住与商业：residential, commercial, retail, industrial，农业与农村：farmland, farmyard, orchard, meadow, allotments，quarry, military，自然与生态公共与休闲：forest, grass, heath, scrub, nature_reserve，park, recreation_ground, cemetery，把面积数量比例都合并，LAD_landuse_group_stats_en.csv

最终版本，有一些苏格兰和威尔士区域

这个用来算面积比例,output_data/LAD_landuse_group_stats_wide.csv

In [1]:
import geopandas as gpd

# 读取两个shp文件
landuse = gpd.read_file('POI/gis_osm_landuse_a_free_1.shp')
lad = gpd.read_file('POI/LAD_MAY_2021_UK_BUC.shp')

print(landuse.shape)
print(lad.shape)
print(landuse.crs)
print(lad.crs)

(2044537, 5)
(374, 9)
EPSG:4326
EPSG:27700


In [7]:
# 转换landuse坐标系为lad的坐标系
landuse = landuse.to_crs(lad.crs)

# 空间连接
joined = gpd.sjoin(landuse, lad, how='inner', predicate='intersects')

# 分组统计
result = (
    joined
    .groupby(['LAD21NM', 'fclass'])
    .size()
    .reset_index(name='count')
)

# 显示结果
print(result.head())

  LAD21NM      fclass  count
0    Adur  allotments     17
1    Adur    cemetery      6
2    Adur  commercial     15
3    Adur    farmland    190
4    Adur    farmyard     21


In [8]:
# 分组统计，保留编号
result = (
    joined
    .groupby(['LAD21CD', 'LAD21NM', 'fclass'])
    .size()
    .reset_index(name='count')
)

# 显示结果
print(result.head())


     LAD21CD     LAD21NM      fclass  count
0  E06000001  Hartlepool  allotments     18
1  E06000001  Hartlepool    cemetery      8
2  E06000001  Hartlepool  commercial     23
3  E06000001  Hartlepool    farmland     29
4  E06000001  Hartlepool    farmyard     43


In [9]:
# 输出数量和面积

import pandas as pd

# 2. 坐标系转换（确保单位为米）
target_crs = 'EPSG:27700'
lad = lad.to_crs(target_crs)
landuse = landuse.to_crs(target_crs)

# 3. 计算土地利用多边形面积
landuse['area'] = landuse.geometry.area  # 单位：平方米

# 4. 空间连接，将 landuse 归入 LAD
joined = gpd.sjoin(landuse, lad, how='inner', predicate='intersects')

# 5. 按 LAD 和 fclass 分组，统计面积和数量
grouped = (
    joined
    .groupby(['LAD21CD', 'LAD21NM', 'fclass'])
    .agg(
        area=('area', 'sum'),
        count=('area', 'size')  # 多边形数量
    )
    .reset_index()
)

# 6. 计算每个 LAD 的 landuse 总面积
total_area = (
    grouped
    .groupby(['LAD21CD', 'LAD21NM'])['area'].sum()
    .reset_index(name='total_area')
)

# 7. 合并分组面积和总面积，算比例
result = grouped.merge(total_area, on=['LAD21CD', 'LAD21NM'])
result['area_ratio'] = result['area'] / result['total_area']

#  显示前几行
print(result.head())



     LAD21CD     LAD21NM      fclass          area  count    total_area  \
0  E06000001  Hartlepool  allotments  3.792989e+05     18  1.042565e+08   
1  E06000001  Hartlepool    cemetery  4.527185e+05      8  1.042565e+08   
2  E06000001  Hartlepool  commercial  1.025941e+06     23  1.042565e+08   
3  E06000001  Hartlepool    farmland  5.655965e+07     29  1.042565e+08   
4  E06000001  Hartlepool    farmyard  5.239244e+05     43  1.042565e+08   

   area_ratio  
0    0.003638  
1    0.004342  
2    0.009841  
3    0.542505  
4    0.005025  


In [11]:

# 1. 读取数据
df = result
# 2. 建立英文分组映射字典


group_map = {
    'residential': 'Residential_and_Commercial',
    'commercial': 'Residential_and_Commercial',
    'retail': 'Residential_and_Commercial',
    'industrial': 'Residential_and_Commercial',
    'farmland': 'Agricultural_and_Rural',
    'farmyard': 'Agricultural_and_Rural',
    'orchard': 'Agricultural_and_Rural',
    'meadow': 'Agricultural_and_Rural',
    'allotments': 'Agricultural_and_Rural',
    'quarry': 'Agricultural_and_Rural',
    'military': 'Agricultural_and_Rural',
    'forest': 'Natural_Ecological_and_Public',
    'grass': 'Natural_Ecological_and_Public',
    'heath': 'Natural_Ecological_and_Public',
    'scrub': 'Natural_Ecological_and_Public',
    'nature_reserve': 'Natural_Ecological_and_Public',
    'park': 'Natural_Ecological_and_Public',
    'recreation_ground': 'Natural_Ecological_and_Public',
    'cemetery': 'Natural_Ecological_and_Public',
}

# 3. 添加分组列
df['group'] = df['fclass'].map(group_map)

# 4. 去除未分组的类型（如有）
df = df.dropna(subset=['group'])

# 5. 按 LAD 和分组汇总面积、数量、比例
grouped = (
    df.groupby(['LAD21CD', 'LAD21NM', 'group'])
    .agg(
        landuse_total_area=('area', 'sum'),
        landuse_total_count=('count', 'sum'),
        landuse_total_ratio=('area_ratio', 'sum')
    )
    .reset_index()
)



#  显示前几行
print(grouped.head())



     LAD21CD        LAD21NM                          group  \
0  E06000001     Hartlepool         Agricultural_and_Rural   
1  E06000001     Hartlepool  Natural_Ecological_and_Public   
2  E06000001     Hartlepool     Residential_and_Commercial   
3  E06000002  Middlesbrough         Agricultural_and_Rural   
4  E06000002  Middlesbrough  Natural_Ecological_and_Public   

   landuse_total_area  landuse_total_count  landuse_total_ratio  
0        5.985082e+07                  115             0.574073  
1        1.821019e+07                  477             0.174667  
2        2.619546e+07                  161             0.251260  
3        1.364052e+07                   44             0.282225  
4        6.062113e+06                  719             0.125426  


In [ ]:


# 读取数据
df = grouped
# 只保留需要的列
df = df[['LAD21CD', 'LAD21NM', 'group', 'landuse_total_ratio']]

# 透视表，landuse_group 展开为列
pivot = df.pivot_table(
    index=['LAD21CD', 'LAD21NM'],
    columns='group',
    values='landuse_total_ratio',
    fill_value=0
).reset_index()

# 保存结果
pivot.to_csv('output_data/LAD_landuse_group_stats_wide.csv', index=False)
